In [138]:
from joblib import  load
from xgboost import XGBRegressor
# Modell laden
xgb_model_loaded = load('xgb_model.joblib')

In [139]:
import json

# Öffne die JSON-Datei im Lesemodus und lade ihren Inhalt
with open('data/test_estates.json', 'r') as json_datei:
    json_inhalt = json.load(json_datei)

# Du kannst nun auf die JSON-Daten zugreifen, z.B. so:
print(json_inhalt)

{'postcode': '57580', 'city': 'Gebhardshain VG', 'estate_types': ['HOUSE'], 'estate_subtypes': ['BUNGALOW'], 'plot_area_size_max': 948, 'living_area_size_max': 96, 'rooms_max': 4, 'construction_year': 1959, 'is_new': False, 'features': ['CELLAR_SHARE', 'BARRIER_FREE', 'FITTED_KITCHEN', 'TERRACE', 'GARAGE', 'PARKING_AREA']}


In [140]:
import pandas as pd
# Erstelle ein DataFrame aus dem JSON-Objekt
df = pd.DataFrame.from_dict(json_inhalt, orient='index').T

# Load json

In [141]:
# Öffne die JSON-Datei im Lesemodus und lade ihren Inhalt
with open('fields/estate_subtypes.json', 'r') as json_datei:
    subtypes = json.load(json_datei)
    
with open('fields/estate_types.json', 'r') as json_datei:
    types = json.load(json_datei)
    
with open('fields/features.json', 'r') as json_datei:
    features = json.load(json_datei)
    
with open('fields/city_list.json', 'r') as json_datei:
    city_types = json.load(json_datei)
   
# Füge Spalten für estate_features hinzu
for feature in features['estate_features']:
    column_name = f'feature_{feature}'.lower()
    df[column_name] = df['features'].apply(lambda x: feature in str(x)) 

# Füge Spalten für estate_types und estate_subtypes hinzu
for estate_type in types['estate_types']:
    column_name = f'estate_type_{estate_type}'.lower()
    df[column_name] = df['estate_types'].apply(lambda x: estate_type in x)

for estate_subtype in subtypes['estate_subtypes']:
    column_name = f'estate_subtype_{estate_subtype}'.lower()
    df[column_name] = df['estate_subtypes'].apply(lambda x: estate_subtype in x)


# Erstellen Sie ein leeres Dictionary, das die neuen Spalten aufnehmen wird
new_columns = {}

for key, city in city_types.items():
    # Erstellung der Spalte für jede Stadt
    # key ist der Spaltenname und city ist der Wert, nach dem wir überprüfen
    new_columns[city] = df['city'].apply(lambda x: city in str(x))

# Konvertieren Sie das Dictionary in einen DataFrame
new_columns_df = pd.DataFrame(new_columns)

# Verbinden Sie den neuen DataFrame mit dem bestehenden DataFrame
df = pd.concat([df, new_columns_df], axis=1)


# Da wir nicht wissen, wie genau die 'features'-Spalte im ursprünglichen DataFrame strukturiert ist,
# gehe ich davon aus, dass es eine Liste von Strings ist, die die verschiedenen Features enthält.
# Wenn dem nicht so ist, müssten Sie diesen Teil an Ihre Datenstruktur anpassen.

# Entfernen der ursprünglichen Spalten, nachdem Sie sie transformiert haben
df.drop(['estate_subtypes', 'estate_types', 'features', 'city'], axis=1, inplace=True)
    


In [142]:
df.head(5)

,postcode,plot_area_size_max,living_area_size_max,rooms_max,construction_year,is_new,feature_wheelchair_accessible,feature_fully_renovated,feature_partly_developed,feature_loggia,...,city_Zülpich_/_Sinzenich,city_im_vorderen_Westerwald,city_meerbusch,city_neuss,city_Ötzingen_/_Sainerholz,city_Übach-Palenberg,city_Übach-Palenberg_/_Boscheln,city_Übach-Palenberg_/_Marienberg,city_Üxheim,city_Üxheim_OT_Niederehe
0,57580,948,96,4,1959,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [143]:
df.columns

Index(['postcode', 'plot_area_size_max', 'living_area_size_max', 'rooms_max',
       'construction_year', 'is_new', 'feature_wheelchair_accessible',
       'feature_fully_renovated', 'feature_partly_developed', 'feature_loggia',
       ...
       'city_Zülpich_/_Sinzenich', 'city_im_vorderen_Westerwald',
       'city_meerbusch', 'city_neuss', 'city_Ötzingen_/_Sainerholz',
       'city_Übach-Palenberg', 'city_Übach-Palenberg_/_Boscheln',
       'city_Übach-Palenberg_/_Marienberg', 'city_Üxheim',
       'city_Üxheim_OT_Niederehe'],
      dtype='object', length=1481)

In [144]:
# Konvertierung der numerischen Spalten von 'object' zu 'float' oder 'int'
df['postcode'] = pd.to_numeric(df['postcode'], errors='coerce')
df['plot_area_size_max'] = pd.to_numeric(df['plot_area_size_max'], errors='coerce')
df['living_area_size_max'] = pd.to_numeric(df['living_area_size_max'], errors='coerce')
df['rooms_max'] = pd.to_numeric(df['rooms_max'], errors='coerce')
df['construction_year'] = pd.to_numeric(df['construction_year'], errors='coerce')

# Konvertierung der Spalte 'is_new' von 'object' zu 'bool', wenn es sich um eine binäre Darstellung handelt
df['is_new'] = df['is_new'].astype(int)

# Nach der Konvertierung empfiehlt es sich, den aktualisierten DataFrame erneut zu überprüfen
print(df.dtypes)

postcode                             int64
plot_area_size_max                   int64
living_area_size_max                 int64
rooms_max                            int64
construction_year                    int64
                                     ...  
city_Übach-Palenberg                  bool
city_Übach-Palenberg_/_Boscheln       bool
city_Übach-Palenberg_/_Marienberg     bool
city_Üxheim                           bool
city_Üxheim_OT_Niederehe              bool
Length: 1481, dtype: object


In [145]:
xgb_model_loaded.predict(df)

array([289058.66], dtype=float32)

In [116]:

# Now, to export this DataFrame to a CSV file, you can use the to_csv() method.
df.to_csv('my_file.csv', index=False)